In [2]:
import ee
import pandas as pd
import matplotlib.pyplot as plt

# 1. Initialize Google Earth Engine
try:
    ee.Initialize()
except Exception as e:
    print("Authentication required...")
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine Initialized.")

# 2. Define Function to Extract Data for Any Location
def get_ndvi_history(city_name, lat, lon, start_year=2000, end_year=2024):
    print(f"\n🌍 Fetching 24-year NDVI history for: {city_name}...")
    
    # Define location and buffer (5km radius around the point)
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(5000) 

    # Load MODIS Dataset (MOD13Q1) - 250m resolution, 16-day composites
    # We choose MODIS because it goes back to the year 2000 (Sentinel only starts 2015)
    dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
                .filter(ee.Filter.date(f'{start_year}-01-01', f'{end_year}-01-01')) \
                .select('NDVI')

    # Function to calculate mean NDVI for the region for each image
    def extract_ndvi(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=250,  # MODIS resolution
            maxPixels=1e9
        )
        # Return date and scaled NDVI (MODIS NDVI is scaled by 0.0001)
        return ee.Feature(None, {
            'date': image.date().format('YYYY-MM-dd'),
            'ndvi': stats.get('NDVI'),
            'city': city_name
        })

    # Fetch data from server
    timeseries = dataset.map(extract_ndvi).getInfo()

    # Convert to DataFrame
    data_list = [feat['properties'] for feat in timeseries['features']]
    df = pd.DataFrame(data_list)

    # Cleaning: Handle missing values and scale NDVI
    if not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df['ndvi'] = df['ndvi'] * 0.0001  # Apply scale factor
        df = df.dropna()
        df = df.sort_values('date')
    
    return df

# 3. Define Your Cities (Add as many as you want)
locations = [
    {"name": "Nyeri", "lat": -0.4192962, "lon": 36.9517005},
    {"name": "Nakuru", "lat": -0.3031, "lon": 36.0800},
    {"name": "Eldoret", "lat": 0.5143, "lon": 35.2698}
]

# 4. Loop Through Cities and Collect Data
all_data = []

for loc in locations:
    city_df = get_ndvi_history(loc["name"], loc["lat"], loc["lon"])
    
    if not city_df.empty:
        # Save individual city csv
        filename = f"{loc['name'].lower()}_ndvi_history.csv"
        city_df.to_csv(filename, index=False)
        print(f"   ✅ Saved {len(city_df)} rows to {filename}")
        
        # Add to main list for plotting
        all_data.append(city_df)
        
        # Plotting individual city
        plt.figure(figsize=(12, 4))
        plt.plot(city_df['date'], city_df['ndvi'], color='green', linewidth=0.5)
        plt.title(f'24-Year Crop Health (NDVI) - {loc["name"]}')
        plt.ylabel('NDVI (Vegetation Vigor)')
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print(f"   ⚠️ No data found for {loc['name']}")

# 5. (Optional) Combine all into one large dataset for ML training
if all_data:
    master_df = pd.concat(all_data)
    master_df.to_csv("all_cities_yield_proxy.csv", index=False)
    print(f"\n🎉 Master dataset created with {len(master_df)} rows ready for Machine Learning.")

Authentication required...



Successfully saved authorization token.


EEException: Project ee-samwelodiek is not registered to use Earth Engine. Visit https://console.cloud.google.com/earth-engine/configuration?project=ee-samwelodiek to register your project. See https://developers.google.com/earth-engine/guides/access for more details. If you have previously registered your project for noncommercial use and lost access, you will need to re-verify your eligibility. See the https://developers.google.com/earth-engine/guides/access#configuring_noncommercial_access for more details.